<a href="https://colab.research.google.com/github/nitinware/GoogleCloabNotebooks/blob/master/ml_models_in_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get -y -qq update

In [2]:
!sudo apt-get -y -qq install postgresql

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../0-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../1-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../2-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Selecting previously unselected pac

In [3]:
!sudo service postgresql start

 * Starting PostgreSQL 10 database server
   ...done.


In [4]:
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

ALTER ROLE


In [5]:
!sudo -u postgres psql -U postgres -c "DROP DATABASE IF EXISTS futurex;"

NOTICE:  database "futurex" does not exist, skipping
DROP DATABASE


In [6]:
!sudo -u postgres psql -U postgres -c "CREATE DATABASE futurex;"

CREATE DATABASE


In [7]:
!ls

create_model_table.sql	sample_data


In [8]:
!PGPASSWORD=postgres psql -q -h localhost -p 5432 -U postgres -d futurex -f create_model_table.sql

psql:create_model_table.sql:1: NOTICE:  table "futurex_model_catalog" does not exist, skipping


In [9]:
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [10]:
connection = psycopg2.connect(user='postgres',
                              password='postgres',
                              host='localhost',
                              database='futurex')

In [11]:
cursor = connection.cursor()

In [12]:
select_query = "SELECT * FROM futurex_model_catalog"

In [13]:
cursor.execute(select_query)

In [14]:
models = cursor.fetchall()

In [15]:
models

[]

In [16]:
!ls

create_model_table.sql	sample_data  storepurchasedata.csv


In [17]:
import numpy as np
import pandas as pd

In [18]:
training_data = pd.read_csv('storepurchasedata.csv')

In [19]:
X = training_data.iloc[:, :-1].values
y = training_data.iloc[:,-1].values

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
# minkowski is for ecledian distance
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [23]:
y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:,1]

In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
cm = confusion_matrix(y_test, y_pred)

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
print(accuracy_score(y_test,y_pred))

0.875


In [28]:
import pickle

In [29]:
pickle_classifier_string = pickle.dumps(classifier)
pickle_sc_string = pickle.dumps(sc)

In [30]:
!ls

create_model_table.sql	sample_data  storepurchasedata.csv


In [31]:
pickle_sc_string

b'\x80\x03csklearn.preprocessing._data\nStandardScaler\nq\x00)\x81q\x01}q\x02(X\t\x00\x00\x00with_meanq\x03\x88X\x08\x00\x00\x00with_stdq\x04\x88X\x04\x00\x00\x00copyq\x05\x88X\x0f\x00\x00\x00n_samples_seen_q\x06cnumpy.core.multiarray\nscalar\nq\x07cnumpy\ndtype\nq\x08X\x02\x00\x00\x00i8q\t\x89\x88\x87q\nRq\x0b(K\x03X\x01\x00\x00\x00<q\x0cNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\rbC\x08 \x00\x00\x00\x00\x00\x00\x00q\x0e\x86q\x0fRq\x10X\x05\x00\x00\x00mean_q\x11cnumpy.core.multiarray\n_reconstruct\nq\x12cnumpy\nndarray\nq\x13K\x00\x85q\x14C\x01bq\x15\x87q\x16Rq\x17(K\x01K\x02\x85q\x18h\x08X\x02\x00\x00\x00f8q\x19\x89\x88\x87q\x1aRq\x1b(K\x03h\x0cNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x1cb\x89C\x10\x00\x00\x00\x00\x00tC@\x00\x00\x00\x00(\x10\xe8@q\x1dtq\x1ebX\x04\x00\x00\x00var_q\x1fh\x12h\x13K\x00\x85q h\x15\x87q!Rq"(K\x01K\x02\x85q#h\x1b\x89C\x10\x00\x00\x00\x00\xb8\xc0e@\x00\x00pl\xd9\xc9\xb6Aq$tq%bX\x06\x00\x00\x00scale_q&h\x12h\x13K\x00\x85q\'h\x15\x87q(Rq)(K\x01K\x02\x85q

In [32]:
model_insert_sql = "INSERT INTO futurex_model_catalog VALUES (%s, %s, %s)"

In [33]:
insert_tuple = (1, 'classifier', pickle_classifier_string)

In [34]:
cursor = connection.cursor()

In [35]:
cursor.execute(model_insert_sql, insert_tuple)

In [36]:
insert_tuple = (2, 'sc', pickle_sc_string)

In [37]:
cursor.execute(model_insert_sql, insert_tuple)

In [38]:
cursor.close()

In [39]:
connection.commit()

In [40]:
cursor = connection.cursor()

In [41]:
select_query = "SELECT * FROM futurex_model_catalog"

In [42]:
models = cursor.execute(select_query)

In [43]:
models = cursor.fetchall()

In [44]:
models

[(1, 'classifier', <memory at 0x7f7d8988b1f0>),
 (2, 'sc', <memory at 0x7f7d8988b940>)]

In [45]:
classifier_from_db = pickle.loads(models[0][2])

In [46]:
scaler_from_db = pickle.loads(models[1][2])

In [47]:
new_pred = classifier_from_db.predict(scaler_from_db.transform(np.array([[40, 2000]])))

In [48]:
new_pred

array([0])

In [49]:
new_pred_proba = classifier_from_db.predict_proba(scaler_from_db.transform(np.array([[40, 2000]])))

In [50]:
new_pred_proba

array([[1., 0.]])

In [51]:
new_pred = classifier_from_db.predict(scaler_from_db.transform(np.array([[42, 50000]])))

In [52]:
print(new_pred)

[1]
